# Biopython Annotated Sequences

## Format conversion

Let's start with the **GenBank AF316817 file** from the pre-session video. 

In [1]:
%%bash
cat ../data/AF316817.gb

LOCUS       AF316817                1126 bp    DNA     linear   VRL 14-JUL-2016
DEFINITION  Influenza A virus (A/Athens/2/98 (H3N2)) hemagglutinin gene,
            partial cds.
ACCESSION   AF316817
VERSION     AF316817.1  GI:12231984
KEYWORDS    .
SOURCE      Influenza A virus (A/Athens/2/1998(H3N2))
  ORGANISM  Influenza A virus (A/Athens/2/1998(H3N2))
            Viruses; ssRNA viruses; ssRNA negative-strand viruses;
            Orthomyxoviridae; Influenzavirus A.
REFERENCE   1  (bases 1 to 1126)
  AUTHORS   Plakokefalos,E.T., Markoulatos,P., Spyrou,N. and Vamvakopoulos,N.
  TITLE     Molecular and phylogenetic analysis of hemagglutinin and
            neuraminidase sequences from recent human influenza type A (H3N2)
            viral isolates in Southern Greece
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1126)
  AUTHORS   Plakokefalos,E.T., Markoulatos,P., Spyrou,N. and Vamvakopoulos,N.
  TITLE     Direct Submission
  JOURNAL   Submitted (26-OCT-2000) Virology, Hellenic Past

Rather than write to file, here we use a SeqIO method called `format` that generates a **string containing the whole file**. This is specifically designed for outputting to a web page. (Clearly, one could save this string to a file, but `format`is not recommended for file output: "using Bio.SeqIO.write() is faster and more general", says the Biopython documentation.)

Let's start by reading the file in and outputting it in **FASTA format**:

In [15]:
from Bio import SeqIO
record = SeqIO.read('../data/AF316817.gb', 'genbank')
print(record.format('fasta'))

>AF316817.1 Influenza A virus (A/Athens/2/98 (H3N2)) hemagglutinin gene, partial cds
ATCCCTTAAAAGCACTCAAGCAGACTACATTTTATGTATGGTTTTCGCTCAAAAACTTCC
CGGAAATGACAACAGCACGGCAACGCTGTGCCTGGGACACCATGCAGTGCCAAACGGAAC
GCTAGTGAAAACAATCACGAATGACCAAATTGAAGTGACTAATGCTACTGAGCTGGTTCA
GAGTTCCTCAACAGGTAGAATATGCGACAGTCCTCACCGAATCCTTGATGGAAAAAACTG
CACACTGATAGATGCTCTATTGGGAGACCCTCATTGTGATGACTTCCAAAATAAGGAATG
GGACCTTTTTGTTGAACGCAGCAAAGCTTACAGCAACTGTTACCCTTATGATGTGCCGGA
TTATGCCTCCCTTAGGTCACTAGTTGCCTCATCCGGCACCCTGGAGTTTAACAATGAAAG
CTTCAATTGGACTGGAGTCGCTCAGAATGGAACAAGCTATGCTTGCAAAAGGAGATCTGT
TAAAAGTTTCTTTAGTAGATTGAATTGGTTGCACAAATTAGAATACAAATATCCAGCACT
GAACGTGACTATGCCAAACAATGACAAATTTGACAAATTGTACATTTGGGGGGTTCACCA
CCCGAGTACGGACAGTGACCAAACCAGCGTATATGTTCAAGCATCAGGGAGAGTCACAGT
CTCTACCAAAAGAAGCCAACAAACTGTAATCCCGAATATCGGATCTAGACCCTGGGTAAG
GGGTGTCTCCAGCAGAATAAGCATCTATTGGACAATAGTAAAACCTGGAGACATACTTTT
GATTAACAGCACAGGGAATCTAATTGCTCCTCGGGGTTACTTCAAAATACGAAGTGGGAA
AAGCTCAATAATGAGGTCAGATGCACCCATTGGCAACTGCAATTCTGAATGCATCACTCC


Here is the conversion to **EMBL format** (much closer to the GenBank format than the FASTA format):

In [9]:
print(record.format('embl'))

ID   AF316817; SV 1; linear; DNA; ; VRL; 1126 BP.
XX
AC   AF316817;
XX
DE   Influenza A virus (A/Athens/2/98 (H3N2)) hemagglutinin gene, partial cds
XX
KW   
XX
OS   Influenza A virus (A/Athens/2/1998(H3N2))
OC   Viruses; ssRNA viruses; ssRNA negative-strand viruses; Orthomyxoviridae;
OC   Influenzavirus A.
XX
RN   [1]
RP   1-1126
RA   Plakokefalos,E.T., Markoulatos,P., Spyrou,N. and Vamvakopoulos,N.;
RT   "Molecular and phylogenetic analysis of hemagglutinin and neuraminidase
RT   sequences from recent human influenza type A (H3N2) viral isolates in
RT   Southern Greece";
RL   Unpublished
XX
RN   [2]
RP   1-1126
RA   Plakokefalos,E.T., Markoulatos,P., Spyrou,N. and Vamvakopoulos,N.;
RT   "Direct Submission";
RL   Submitted (26-OCT-2000) Virology, Hellenic Pasteur Institute, 127
RL   Vasilissis Sofias Ave., Athens 11521, Greece
XX
FH   Key             Location/Qualifiers
FH
FT   source          1..1126
FT                   /organism="Influenza A virus (A/Athens/2/1998(H3N2))"
FT       

## Generating random mutated sequences

Here we're going to take the original GenBank record and make new records that are variants of the original (but trimmed in length so we can see them easily on the screen). We will do this by adding some **random mutations, insertions and deletions**:

In [67]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from random import randint

# NOTE: you can change these parameters if you want, 
# but I would avoid generating hundreds of sequences,
# as we will subsequently align them and don't want  
# to overload the departmental gateway server!
number_of_new_sequences = 15
seq_length = 50 
max_number_of_mutations = 20
max_number_of_insertions = 10
max_number_of_deletions = 10

all_seqs = []
bases = "ACGT"
for i in range(0, number_of_new_sequences):
    # create a mutable sequence because we are 
    # going to make multiple modifications
    new_seq = record.seq.tomutable() # mutable and sequence class - record taken from the GenBank record above
    new_seq = new_seq[:seq_length] # slice sequence to 50 as specified
    
    # mutations
    for j in range(0, max_number_of_mutations): # runs up to number of mutation occurances specified
        pos = randint(0, seq_length - 1) # random index number between the start index and the final index
        new_seq[pos] = bases[randint(0, len(bases) - 1)] # new_seq[pos] - random mutation index, randomly selects index from the bases string to replace with
    
    # insertions
    for j in range(0, max_number_of_insertions):
        if randint(0, 2) % 2: # not sure - ask in session
            continue
        pos = randint(0, seq_length - 1)
        new_seq = new_seq[:pos] + bases[randint(0, len(bases) - 1)] + new_seq[pos:] # slice before position, insertion of 1 base, slice after position

    # deletions
    for j in range(0, max_number_of_insertions):
        if randint(0, 2) % 2:
            continue
        pos = randint(0, seq_length - 1)
        del new_seq[pos]
    all_seqs.append(new_seq)
    
# print out the sequences
for s in all_seqs:
    print(s, len(s))

GTCCATAAAAGGGACCTCAAAAGGGTATTCAATTTATTGATTTTCTCGCTC 51
TCGCTTAACCTACCTCAATCAGATGACATTTTAAGTTGGTTTTTGTC 47
ACTCCCAGAGCGGCTGTATAGAGCCTAGATTGTAGAGCGTTCTCCGCTG 49
ACCCTTGACAGGCATGCAGAGCAAACTCAGGTTTTACTTATTTCAAC 47
AGTCCCTTGTAAGGTTCAAGCAGGCCATTTTATGTGGATTCCTGGTC 47
ACCCATAAAGCGCCTCCAAAGGACTACTTCCTTGGATTCATTTTAGCTC 49
ATACTTGCAAGCTCTCAGTTTTAGGCTCCATTTATGAATACTTATCTG 48
ATACCATGAACAGCACCAACTCACGACTACGTTCCAACACGACGTCAATT 50
CCATGTAAAAAGCTCCTGAGGTGACTAACATTCCTATCTTTAGCTCGCTC 50
GTCCCTAATAGTACTGAAAGCCTGCTCGCACGCTTATGTTAGTTCTC 47
ACTCCCCTAATCTAATCTAGCCTGAACATTTTGAATGCCTTTTACTCTTC 50
ATCCTACAAAACCGTCGTGAGACTAACATTTAATAGGGGCATAATC 46
TTCATAAAAGCCTAAGCAGCCATCCTTGGTTGTATCGAGTTCGTCC 46
CTCTCCGAATAGGACTTAAAGCGGCTACAACATGATGGATGTTTCTCTC 49
ATCCATTGCCAGCACCAATAACTACATTTACTGGAAGGACTTCTC 45


Now let's turn these sequences into records with identifiers and some minimal annotation, and then save them in a **FASTA format file**:

In [83]:
all_records = []
all_records.append(record)
for i, s in enumerate(all_seqs):
    this_record = SeqRecord(
        s,
        id= str(i + 1),
        name='',
        description='random mutations/insertions/deletions',  # not sure where 'IUPACAmbiguousDNA()' comes from if you print the output?
    )
    all_records.append(this_record)

#print(all_records)
SeqIO.write(all_records, 'random_seqs.fasta', 'fasta')
print(all_records)

[SeqRecord(seq=Seq('ATCCCTTAAAAGCACTCAAGCAGACTACATTTTATGTATGGTTTTCGCTCAAAA...TGA', IUPACAmbiguousDNA()), id='AF316817.1', name='AF316817', description='Influenza A virus (A/Athens/2/98 (H3N2)) hemagglutinin gene, partial cds', dbxrefs=[]), SeqRecord(seq=MutableSeq('GTCCATAAAAGGGACCTCAAAAGGGTATTCAATTTATTGATTTTCTCGCTC', IUPACAmbiguousDNA()), id='1', name='', description='random mutations/insertions/deletions', dbxrefs=[]), SeqRecord(seq=MutableSeq('TCGCTTAACCTACCTCAATCAGATGACATTTTAAGTTGGTTTTTGTC', IUPACAmbiguousDNA()), id='2', name='', description='random mutations/insertions/deletions', dbxrefs=[]), SeqRecord(seq=MutableSeq('ACTCCCAGAGCGGCTGTATAGAGCCTAGATTGTAGAGCGTTCTCCGCTG', IUPACAmbiguousDNA()), id='3', name='', description='random mutations/insertions/deletions', dbxrefs=[]), SeqRecord(seq=MutableSeq('ACCCTTGACAGGCATGCAGAGCAAACTCAGGTTTTACTTATTTCAAC', IUPACAmbiguousDNA()), id='4', name='', description='random mutations/insertions/deletions', dbxrefs=[]), SeqRecord(seq=MutableSeq('AGTC

Note the new file that has appeared in the directory listing (you can also run `ls` to see this). You can inspect the content by opening our new file with the text editor (or see the first few lines using `head`):

In [70]:
!ls

py5_1_sequences.ipynb	py5_practical.ipynb
py5_2_annotation.ipynb	random_seqs.fasta


In [71]:
!head -5 random_seqs.fasta

>AF316817.1 Influenza A virus (A/Athens/2/98 (H3N2)) hemagglutinin gene, partial cds
ATCCCTTAAAAGCACTCAAGCAGACTACATTTTATGTATGGTTTTCGCTCAAAAACTTCC
CGGAAATGACAACAGCACGGCAACGCTGTGCCTGGGACACCATGCAGTGCCAAACGGAAC
GCTAGTGAAAACAATCACGAATGACCAAATTGAAGTGACTAATGCTACTGAGCTGGTTCA
GAGTTCCTCAACAGGTAGAATATGCGACAGTCCTCACCGAATCCTTGATGGAAAAAACTG


## Generating an alignment

Now let's create a **ClustalW alignment** from the randomly mutated sequences we have just created:

In [84]:
from Bio.Align.Applications import ClustalwCommandline

cline = ClustalwCommandline('clustalw2', infile='random_seqs.fasta')
results = cline()

In [85]:
# uncomment this to see a formatted version of the command line output
for l in results:
    print(l)




 CLUSTAL 2.1 Multiple Sequence Alignments


Sequence format is Pearson
Sequence 1: AF316817.1  1126 bp
Sequence 2: 1             51 bp
Sequence 3: 2             47 bp
Sequence 4: 3             49 bp
Sequence 5: 4             47 bp
Sequence 6: 5             47 bp
Sequence 7: 6             49 bp
Sequence 8: 7             48 bp
Sequence 9: 8             50 bp
Sequence 10: 9             50 bp
Sequence 11: 10            47 bp
Sequence 12: 11            50 bp
Sequence 13: 12            46 bp
Sequence 14: 13            46 bp
Sequence 15: 14            49 bp
Sequence 16: 15            45 bp
Start of Pairwise alignments
Aligning...

Sequences (1:2) Aligned. Score:  60
Sequences (1:3) Aligned. Score:  72
Sequences (1:4) Aligned. Score:  22
Sequences (1:5) Aligned. Score:  36
Sequences (1:6) Aligned. Score:  59
Sequences (1:7) Aligned. Score:  59
Sequences (1:8) Aligned. Score:  27
Sequences (1:9) Aligned. Score:  42
Sequences (1:10) Aligned. Score:  22
Sequences (1:11) Aligned. Score:  27
Seq

The alignment was generated via the command-line and saved in a file with the `.aln` extension. Let's open it and **print it out**:

In [86]:
from Bio import AlignIO
alignment = AlignIO.read('random_seqs.aln', 'clustal')
print(alignment)

SingleLetterAlphabet() alignment with 16 rows and 1132 columns
------ATCCCTTAAAAGCACTCAAGCAGACTACATTTTATGTA...TGA AF316817.1
------ATCCATTGCCAGCACCAA---TAACTACATTTACTGGA...--- 15
-------TCGCTTAACCTACCTCAATCAGATGACATTTTAAGTT...--- 2
-----AGTCCCTTG-TAAGGTTCAAGCAGGC--CATTTTATGTG...--- 5
---CCATGTAAAAAGCTCCTGAGGTGACTAACATTCCTATCTTT...--- 9
-----ATCCTACAAAACCGTCGTGAGACTAACATT--TAATAGG...--- 12
ATACCATG-AACAGCACCAACTCACGACTA-CGTTC-CAACACG...--- 8
----GTCCATAAAAGGGACCTCAAAAGGGTATTCAATTT-ATTG...--- 1
----ACCCATAAA-GCGCCTCCAAAGGACTACTTCCTTGGATTC...--- 6
----GTCCCTAATAGTA---CTGAAAGCCTGCTCGCACGCTTAT...--- 10
---CTCTCCGAATAGGA---CTTAAAGCG-GCTACAACATGATG...--- 14
----ATACTTGCAAGCT---CTCAGTTTTAGGCTCCAT-TTATG...--- 7
------ACTCCCCTAAT---CTAA--TCTAGCCTGAACATTTTG...--- 11
------ACTCCCAGAGCGGCTGTATAG--AGCCTAGATTGTAGA...--- 3
--------TTCATAAAAGCCTAAGCAGCCATCCTTGGTTGTATC...--- 13
------ACCCTTGACAGGCATGCAGAGCAAACTCAGGTTTTACT...--- 4


Note that not everything goes smoothly. Here I would like to convert the alignment to **PHYLIP format**, but this leads to **name clashes**, because PHYLIP truncates the identifier (see last line of error message generated below).

Consider modifiying the code above to resolve this issue!

In [87]:
print(alignment.format('phylip'))

 16 1132
AF316817.1 ------ATCC CTTAAAAGCA CTCAAGCAGA CTACATTTTA TGTATGGTTT
15         ------ATCC ATTGCCAGCA CCAA---TAA CTACATTTAC TGGAAGGACT
2          -------TCG CTTAACCTAC CTCAATCAGA TGACATTTTA AGTTGGTTTT
5          -----AGTCC CTTG-TAAGG TTCAAGCAGG C--CATTTTA TGTGGATTCC
9          ---CCATGTA AAAAGCTCCT GAGGTGACTA ACATTCCTAT CTTTAGCTCG
12         -----ATCCT ACAAAACCGT CGTGAGACTA ACATT--TAA TAGGGGCATA
8          ATACCATG-A ACAGCACCAA CTCACGACTA -CGTTC-CAA CACGACGTCA
1          ----GTCCAT AAAAGGGACC TCAAAAGGGT ATTCAATTT- ATTGATTTTC
6          ----ACCCAT AAA-GCGCCT CCAAAGGACT ACTTCCTTGG ATTCATTTTA
10         ----GTCCCT AATAGTA--- CTGAAAGCCT GCTCGCACGC TTATGTTAGT
14         ---CTCTCCG AATAGGA--- CTTAAAGCG- GCTACAACAT GATGGATGTT
7          ----ATACTT GCAAGCT--- CTCAGTTTTA GGCTCCAT-T TATGAATACT
11         ------ACTC CCCTAAT--- CTAA--TCTA GCCTGAACAT TTTGAATGCC
3          ------ACTC CCAGAGCGGC TGTATAG--A GCCTAGATTG TAGAGCGTTC
13         --------TT CATAAAAGCC TAAGCAGCCA TCCTTGGTTG TATCGAGTTC
4